# 02. LangFuse Observations & Debugging

## 📋 Mục tiêu học tập

Notebook này sẽ hướng dẫn bạn:
- Hiểu cách sử dụng LangFuse để quan sát và debug ứng dụng LLM
- Phân tích các Span, Event, và Observation trong LangFuse UI
- Theo dõi luồng hoạt động của LLM chains và RAG pipelines
- Xác định và khắc phục các lỗi tiềm ẩn trong ứng dụng LLM
- Đo lường hiệu suất (latency, token usage) và tối ưu hóa

## 🔍 Giới thiệu về LangFuse Debugging

LangFuse cung cấp khả năng quan sát chi tiết cho ứng dụng LLM thông qua:

### 📊 Các thành phần quan sát chính:
- **Traces**: Toàn bộ luồng thực thi của một request
- **Spans**: Các bước con trong một trace (LLM calls, retrieval, processing)
- **Events**: Các sự kiện đơn lẻ (logging, errors, milestones)
- **Observations**: Tổng hợp tất cả hoạt động có thể quan sát được

### 🎯 Lợi ích của debugging với LangFuse:
- **Visibility**: Nhìn thấy toàn bộ luồng hoạt động của LLM
- **Performance**: Đo lường latency và token usage chi tiết
- **Error Detection**: Phát hiện lỗi và bottlenecks
- **Quality Assurance**: Đánh giá chất lượng output
- **Cost Optimization**: Theo dõi chi phí sử dụng LLM

## ⚙️ Cài đặt & Cấu hình

Nhắc lại cấu hình LangFuse từ notebook trước:

In [ ]:
import os
from langfuse import Langfuse
from langfuse.callback import CallbackHandler
from langfuse.decorators import observe, langfuse_context

# Cấu hình LangFuse
langfuse = Langfuse(
    public_key=os.getenv("LANGFUSE_PUBLIC_KEY", "pk-lf-your-public-key"),
    secret_key=os.getenv("LANGFUSE_SECRET_KEY", "sk-lf-your-secret-key"),
    host=os.getenv("LANGFUSE_HOST", "http://localhost:3000")
)

# Tạo callback handler
langfuse_handler = CallbackHandler(
    public_key=os.getenv("LANGFUSE_PUBLIC_KEY", "pk-lf-your-public-key"),
    secret_key=os.getenv("LANGFUSE_SECRET_KEY", "sk-lf-your-secret-key"),
    host=os.getenv("LANGFUSE_HOST", "http://localhost:3000")
)

print("✅ LangFuse configured successfully!")

## 📦 Import các thư viện cần thiết

In [ ]:
import time
import json
from typing import List, Dict, Any
from datetime import datetime

# LangChain imports
from langchain_anthropic import ChatAnthropic
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain.chains import LLMChain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings

print("📚 All libraries imported successfully!")

## 🤖 Cấu hình LLM với LangFuse tracing

In [ ]:
# Khởi tạo ChatAnthropic với LangFuse callback
llm = ChatAnthropic(
    model="claude-3-sonnet-20240229",
    temperature=0.7,
    max_tokens=1000,
    callbacks=[langfuse_handler]
)

print("🤖 LLM initialized with LangFuse tracing!")

## 🚨 VÍ DỤ 1: Chain có lỗi tiềm ẩn

Chúng ta sẽ tạo một chain có thể gặp lỗi để thấy cách LangFuse giúp debug:

In [ ]:
@observe(name="problematic_chain")
def problematic_math_chain(question: str) -> str:
    """
    Chain giải toán có thể gặp lỗi với một số input nhất định
    """
    # Tạo span cho preprocessing
    langfuse_context.update_current_observation(
        name="math_preprocessing",
        input={"raw_question": question}
    )
    
    # Preprocessing - có thể gây lỗi nếu input không phù hợp
    if len(question.strip()) < 5:
        langfuse_context.update_current_observation(
            level="ERROR",
            status_message="Input quá ngắn để xử lý"
        )
        raise ValueError("Question too short for processing")
    
    # Tạo prompt cho toán học
    math_prompt = ChatPromptTemplate.from_messages([
        ("system", "Bạn là một trợ lý toán học thông minh. Hãy giải bài toán một cách chi tiết và chính xác."),
        ("human", "Câu hỏi: {question}\n\nHãy giải thích từng bước một cách rõ ràng.")
    ])
    
    # Tạo chain
    chain = math_prompt | llm | StrOutputParser()
    
    # Thực thi với error handling
    try:
        result = chain.invoke(
            {"question": question},
            config={"callbacks": [langfuse_handler]}
        )
        
        # Log thành công
        langfuse_context.update_current_observation(
            output={"answer": result, "status": "success"}
        )
        
        return result
        
    except Exception as e:
        # Log lỗi
        langfuse_context.update_current_observation(
            level="ERROR",
            status_message=f"Chain execution failed: {str(e)}",
            output={"error": str(e)}
        )
        raise e

print("🔧 Problematic chain function created!")

### 🧪 Test chain với các input khác nhau

In [ ]:
# Test cases
test_cases = [
    "Tính 15 + 27 = ?",  # Câu hỏi bình thường
    "2+2",  # Câu hỏi ngắn - sẽ gây lỗi
    "Giải phương trình x² - 5x + 6 = 0",  # Câu hỏi phức tạp
    "Hi"  # Input quá ngắn - sẽ gây lỗi
]

print("🧪 Testing problematic chain with different inputs...\n")

for i, question in enumerate(test_cases, 1):
    print(f"Test {i}: {question}")
    print("-" * 50)
    
    try:
        result = problematic_math_chain(question)
        print(f"✅ Success: {result[:100]}..." if len(result) > 100 else f"✅ Success: {result}")
    except Exception as e:
        print(f"❌ Error: {e}")
    
    print("\n")
    time.sleep(1)  # Delay để dễ theo dõi trên LangFuse

print("🔍 Check LangFuse UI để xem traces và errors!")

## 🔍 VÍ DỤ 2: RAG Pipeline với detailed tracing

Tạo một RAG pipeline đơn giản với tracing chi tiết cho việc retrieval và generation:

In [ ]:
# Tạo sample documents
sample_documents = [
    "Python là một ngôn ngữ lập trình cấp cao, dễ học và mạnh mẽ. Python được sử dụng rộng rãi trong khoa học dữ liệu, phát triển web, và trí tuệ nhân tạo.",
    "Machine Learning là một nhánh của trí tuệ nhân tạo cho phép máy tính học từ dữ liệu mà không cần được lập trình cụ thể cho từng tác vụ.",
    "LangChain là một framework mạnh mẽ để xây dựng ứng dụng với các mô hình ngôn ngữ lớn (LLM). Nó cung cấp các công cụ để tạo chains, agents và các thành phần khác.",
    "RAG (Retrieval-Augmented Generation) là một kỹ thuật kết hợp việc truy xuất thông tin từ cơ sở dữ liệu với khả năng sinh văn bản của LLM.",
    "Vector databases như FAISS, Pinecone được sử dụng để lưu trữ và tìm kiếm embeddings một cách hiệu quả trong các hệ thống RAG."
]

print("📄 Sample documents created!")

In [ ]:
@observe(name="setup_rag_pipeline")
def setup_rag_pipeline(documents: List[str]):
    """
    Thiết lập RAG pipeline với detailed tracing
    """
    # Tạo span cho document processing
    langfuse_context.update_current_observation(
        name="document_processing",
        input={"num_documents": len(documents)}
    )
    
    # Text splitting
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=200,
        chunk_overlap=50
    )
    
    splits = text_splitter.create_documents(documents)
    
    # Tạo embeddings
    embeddings = HuggingFaceEmbeddings(
        model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
    )
    
    # Tạo vector store
    vectorstore = FAISS.from_documents(splits, embeddings)
    
    # Log thông tin về vector store
    langfuse_context.update_current_observation(
        output={
            "num_chunks": len(splits),
            "embedding_model": "paraphrase-multilingual-MiniLM-L12-v2",
            "vectorstore_type": "FAISS"
        }
    )
    
    return vectorstore

# Setup RAG pipeline
vectorstore = setup_rag_pipeline(sample_documents)
print("🔧 RAG pipeline setup completed!")

In [ ]:
@observe(name="rag_query")
def rag_query(question: str, vectorstore, k: int = 3):
    """
    Thực hiện RAG query với detailed tracing
    """
    start_time = time.time()
    
    # Tạo span cho retrieval
    langfuse_context.update_current_observation(
        name="retrieval_phase",
        input={"question": question, "k": k}
    )
    
    # Retrieval
    retriever = vectorstore.as_retriever(search_kwargs={"k": k})
    retrieved_docs = retriever.get_relevant_documents(question)
    
    # Log retrieval results
    retrieval_time = time.time() - start_time
    retrieval_results = [
        {"content": doc.page_content[:100] + "...", "metadata": doc.metadata}
        for doc in retrieved_docs
    ]
    
    langfuse_context.update_current_observation(
        name="retrieval_results",
        output={
            "num_retrieved": len(retrieved_docs),
            "retrieval_time_ms": round(retrieval_time * 1000, 2),
            "documents": retrieval_results
        }
    )
    
    # Tạo context từ retrieved documents
    context = "\n\n".join([doc.page_content for doc in retrieved_docs])
    
    # Tạo span cho generation
    generation_start = time.time()
    
    # RAG prompt
    rag_prompt = ChatPromptTemplate.from_messages([
        ("system", "Bạn là một trợ lý AI thông minh. Hãy trả lời câu hỏi dựa trên thông tin được cung cấp."),
        ("human", """Thông tin tham khảo:
{context}

Câu hỏi: {question}

Hãy trả lời câu hỏi dựa trên thông tin tham khảo trên. Nếu thông tin không đủ, hãy nói rõ điều đó.""")
    ])
    
    # Tạo chain
    rag_chain = rag_prompt | llm | StrOutputParser()
    
    # Generation
    response = rag_chain.invoke(
        {"context": context, "question": question},
        config={"callbacks": [langfuse_handler]}
    )
    
    # Log generation results
    generation_time = time.time() - generation_start
    total_time = time.time() - start_time
    
    langfuse_context.update_current_observation(
        name="generation_results",
        output={
            "response": response,
            "generation_time_ms": round(generation_time * 1000, 2),
            "total_time_ms": round(total_time * 1000, 2),
            "context_length": len(context)
        }
    )
    
    return {
        "answer": response,
        "retrieved_docs": retrieved_docs,
        "context": context,
        "metrics": {
            "retrieval_time_ms": round(retrieval_time * 1000, 2),
            "generation_time_ms": round(generation_time * 1000, 2),
            "total_time_ms": round(total_time * 1000, 2)
        }
    }

print("🔍 RAG query function created!")

### 🧪 Test RAG pipeline với các câu hỏi khác nhau

In [ ]:
# Test queries
test_queries = [
    "Python được sử dụng để làm gì?",
    "RAG là gì và hoạt động như thế nào?",
    "Vector database có vai trò gì trong RAG?",
    "Blockchain là gì?"  # Câu hỏi ngoài phạm vi tài liệu
]

print("🧪 Testing RAG pipeline with different queries...\n")

for i, query in enumerate(test_queries, 1):
    print(f"Query {i}: {query}")
    print("=" * 60)
    
    try:
        result = rag_query(query, vectorstore)
        
        print(f"📝 Answer: {result['answer'][:200]}..." if len(result['answer']) > 200 else f"📝 Answer: {result['answer']}")
        print(f"📊 Metrics: {result['metrics']}")
        print(f"📚 Retrieved {len(result['retrieved_docs'])} documents")
        
    except Exception as e:
        print(f"❌ Error: {e}")
    
    print("\n")
    time.sleep(2)  # Delay để dễ theo dõi trên LangFuse

print("🔍 Check LangFuse UI để xem RAG traces chi tiết!")

## 📊 Tạo trace với custom metadata và tags

In [ ]:
@observe(name="advanced_llm_call")
def advanced_llm_call_with_metadata(prompt: str, user_id: str = "demo_user"):
    """
    LLM call với metadata và tags chi tiết
    """
    # Cập nhật observation với metadata
    langfuse_context.update_current_observation(
        user_id=user_id,
        session_id=f"session_{datetime.now().strftime('%Y%m%d_%H%M%S')}",
        metadata={
            "prompt_length": len(prompt),
            "timestamp": datetime.now().isoformat(),
            "model_version": "claude-3-sonnet-20240229",
            "temperature": 0.7
        },
        tags=["advanced_call", "with_metadata", "demo"]
    )
    
    # Tạo messages
    messages = [
        SystemMessage(content="Bạn là một trợ lý AI hữu ích và thông minh."),
        HumanMessage(content=prompt)
    ]
    
    # Call LLM
    response = llm.invoke(
        messages,
        config={"callbacks": [langfuse_handler]}
    )
    
    # Log response metadata
    langfuse_context.update_current_observation(
        output={
            "response": response.content,
            "response_length": len(response.content),
            "token_usage": getattr(response, 'usage_metadata', {})
        }
    )
    
    return response.content

# Test với metadata
response = advanced_llm_call_with_metadata(
    "Giải thích khái niệm Machine Learning một cách đơn giản",
    user_id="user_123"
)

print("📝 Response with metadata:")
print(response[:300] + "..." if len(response) > 300 else response)

## 🎯 Tạo scores và evaluations

In [ ]:
@observe(name="evaluated_llm_call")
def evaluated_llm_call(question: str, expected_topics: List[str]):
    """
    LLM call với evaluation scores
    """
    start_time = time.time()
    
    # Call LLM
    response = llm.invoke(
        [HumanMessage(content=question)],
        config={"callbacks": [langfuse_handler]}
    )
    
    response_time = time.time() - start_time
    response_text = response.content
    
    # Simple evaluation scores
    relevance_score = sum(1 for topic in expected_topics if topic.lower() in response_text.lower()) / len(expected_topics)
    length_score = min(len(response_text) / 500, 1.0)  # Điểm dựa trên độ dài
    speed_score = max(0, 1 - (response_time / 10))  # Điểm dựa trên tốc độ
    
    # Tạo scores trong LangFuse
    langfuse_context.score_current_observation(
        name="relevance",
        value=relevance_score,
        comment=f"Matches {int(relevance_score * len(expected_topics))}/{len(expected_topics)} expected topics"
    )
    
    langfuse_context.score_current_observation(
        name="response_length",
        value=length_score,
        comment=f"Response length: {len(response_text)} characters"
    )
    
    langfuse_context.score_current_observation(
        name="response_speed",
        value=speed_score,
        comment=f"Response time: {response_time:.2f} seconds"
    )
    
    # Overall score
    overall_score = (relevance_score + length_score + speed_score) / 3
    langfuse_context.score_current_observation(
        name="overall_quality",
        value=overall_score,
        comment="Average of relevance, length, and speed scores"
    )
    
    return {
        "response": response_text,
        "scores": {
            "relevance": relevance_score,
            "length": length_score,
            "speed": speed_score,
            "overall": overall_score
        },
        "metrics": {
            "response_time": response_time,
            "response_length": len(response_text)
        }
    }

# Test với evaluation
result = evaluated_llm_call(
    "Giải thích về Deep Learning và ứng dụng của nó",
    expected_topics=["neural networks", "training", "applications", "data"]
)

print("📊 Evaluation Results:")
print(f"Scores: {result['scores']}")
print(f"Metrics: {result['metrics']}")
print(f"Response: {result['response'][:200]}...")

## 🔍 Phân tích và Debug trên LangFuse UI

### 📱 Cách sử dụng LangFuse UI để phân tích:

1. **Mở LangFuse Dashboard**: Truy cập `http://localhost:3000`

2. **Xem Traces**:
   - Vào tab "Traces" để xem tất cả execution traces
   - Click vào trace để xem chi tiết
   - Quan sát timeline và nested spans

3. **Phân tích Observations**:
   - **Input/Output**: Xem dữ liệu đầu vào và đầu ra
   - **Latency**: Đo thời gian thực thi từng bước
   - **Token Usage**: Theo dõi số token được sử dụng
   - **Errors**: Xác định lỗi và nguyên nhân

4. **Metrics Dashboard**:
   - Xem tổng quan về performance
   - Theo dõi cost và usage
   - Phân tích trends theo thời gian

5. **Scores và Evaluations**:
   - Xem scores cho từng trace
   - So sánh quality metrics
   - Tạo evaluations tự động

### 🎯 Các điểm cần chú ý khi debug:

- **Latency Bottlenecks**: Spans nào mất thời gian nhất?
- **Error Patterns**: Lỗi xuất hiện ở đâu và tần suất?
- **Token Usage**: Chi phí có tối ưu không?
- **Quality Scores**: Output có đạt chất lượng mong muốn?
- **User Experience**: Response time có chấp nhận được không?

## 📈 Tạo comprehensive trace cho production debugging

In [ ]:
@observe(name="production_rag_pipeline")
def production_rag_pipeline(query: str, user_id: str, session_id: str):
    """
    Production-ready RAG pipeline với comprehensive tracing
    """
    pipeline_start = time.time()
    
    # Set session và user context
    langfuse_context.update_current_observation(
        user_id=user_id,
        session_id=session_id,
        metadata={
            "pipeline_version": "v1.0.0",
            "environment": "development",
            "timestamp": datetime.now().isoformat()
        },
        tags=["production", "rag", "comprehensive"]
    )
    
    try:
        # Step 1: Query preprocessing
        with langfuse_context.observe(name="query_preprocessing") as preprocessing_span:
            preprocessed_query = query.strip().lower()
            preprocessing_span.update(
                input={"raw_query": query},
                output={"preprocessed_query": preprocessed_query},
                metadata={"preprocessing_time": time.time() - pipeline_start}
            )
        
        # Step 2: Retrieval
        retrieval_start = time.time()
        with langfuse_context.observe(name="document_retrieval") as retrieval_span:
            retriever = vectorstore.as_retriever(search_kwargs={"k": 3})
            retrieved_docs = retriever.get_relevant_documents(preprocessed_query)
            
            retrieval_time = time.time() - retrieval_start
            retrieval_span.update(
                input={"query": preprocessed_query, "k": 3},
                output={
                    "num_documents": len(retrieved_docs),
                    "documents": [doc.page_content[:100] for doc in retrieved_docs]
                },
                metadata={"retrieval_time_ms": round(retrieval_time * 1000, 2)}
            )
            
            # Score retrieval quality
            retrieval_span.score(
                name="retrieval_quality",
                value=min(len(retrieved_docs) / 3, 1.0),
                comment=f"Retrieved {len(retrieved_docs)} out of 3 requested documents"
            )
        
        # Step 3: Context preparation
        with langfuse_context.observe(name="context_preparation") as context_span:
            context = "\n\n".join([doc.page_content for doc in retrieved_docs])
            context_span.update(
                input={"documents": [doc.page_content for doc in retrieved_docs]},
                output={"context_length": len(context)},
                metadata={"num_documents_used": len(retrieved_docs)}
            )
        
        # Step 4: Generation
        generation_start = time.time()
        with langfuse_context.observe(name="answer_generation") as generation_span:
            rag_prompt = ChatPromptTemplate.from_messages([
                ("system", "Bạn là một trợ lý thông minh. Trả lời dựa trên thông tin được cung cấp."),
                ("human", "Thông tin: {context}\n\nCâu hỏi: {question}\n\nTrả lời:")
            ])
            
            chain = rag_prompt | llm | StrOutputParser()
            answer = chain.invoke(
                {"context": context, "question": query},
                config={"callbacks": [langfuse_handler]}
            )
            
            generation_time = time.time() - generation_start
            generation_span.update(
                input={"context_length": len(context), "question": query},
                output={"answer": answer, "answer_length": len(answer)},
                metadata={"generation_time_ms": round(generation_time * 1000, 2)}
            )
            
            # Score generation quality
            generation_span.score(
                name="answer_completeness",
                value=min(len(answer) / 200, 1.0),
                comment=f"Answer length: {len(answer)} characters"
            )
        
        # Step 5: Post-processing & final results
        total_time = time.time() - pipeline_start
        
        result = {
            "answer": answer,
            "retrieved_docs": len(retrieved_docs),
            "context_length": len(context),
            "total_time_ms": round(total_time * 1000, 2),
            "retrieval_time_ms": round(retrieval_time * 1000, 2),
            "generation_time_ms": round(generation_time * 1000, 2)
        }
        
        # Overall pipeline score
        langfuse_context.score_current_observation(
            name="pipeline_performance",
            value=max(0, 1 - (total_time / 30)),  # Penalty for slow responses
            comment=f"Total pipeline time: {total_time:.2f}s"
        )
        
        langfuse_context.update_current_observation(
            output=result,
            status_message="Pipeline completed successfully"
        )
        
        return result
        
    except Exception as e:
        error_time = time.time() - pipeline_start
        langfuse_context.update_current_observation(
            level="ERROR",
            status_message=f"Pipeline failed after {error_time:.2f}s: {str(e)}",
            output={"error": str(e), "error_type": type(e).__name__}
        )
        raise e

# Test production pipeline
production_result = production_rag_pipeline(
    query="Giải thích về LangChain và các ứng dụng của nó",
    user_id="prod_user_001",
    session_id="session_prod_20241224"
)

print("🚀 Production Pipeline Results:")
print(json.dumps(production_result, ensure_ascii=False, indent=2))

## 🔧 Utility functions cho debugging

In [ ]:
def get_trace_url(trace_id: str) -> str:
    """
    Tạo URL để xem trace trên LangFuse UI
    """
    base_url = os.getenv("LANGFUSE_HOST", "http://localhost:3000")
    return f"{base_url}/trace/{trace_id}"

def flush_langfuse():
    """
    Flush tất cả pending traces to LangFuse
    """
    langfuse.flush()
    print("✅ All traces flushed to LangFuse!")

def create_debug_session(session_name: str):
    """
    Tạo debug session với metadata
    """
    return langfuse.trace(
        name=f"debug_session_{session_name}",
        metadata={
            "session_type": "debug",
            "created_at": datetime.now().isoformat(),
            "notebook": "02_LangFuse_Observations_Debugging"
        },
        tags=["debug", "notebook", session_name]
    )

# Flush tất cả traces
flush_langfuse()

print("🔧 Debug utilities ready!")
print("🌐 LangFuse UI: http://localhost:3000")
print("📊 Check your traces and observations in the UI!")

## 🎯 Kết luận & Bước tiếp theo

### 📚 Nội dung đã học:

1. **Quan sát chi tiết**: Sử dụng LangFuse để theo dõi từng bước trong LLM pipeline
2. **Error Detection**: Phát hiện và debug lỗi với detailed tracing
3. **Performance Monitoring**: Đo lường latency, token usage, và quality metrics
4. **RAG Debugging**: Trace chi tiết cho retrieval và generation phases
5. **Production Ready**: Tạo comprehensive traces cho production systems
6. **Evaluation Scores**: Tích hợp scoring và evaluation vào traces

### 🔍 Key Benefits của LangFuse Debugging:

- **🎯 Visibility**: Nhìn thấy toàn bộ luồng hoạt động
- **⚡ Performance**: Tối ưu hóa speed và cost
- **🐛 Debug**: Nhanh chóng xác định và sửa lỗi
- **📊 Analytics**: Phân tích patterns và trends
- **🔄 Iteration**: Cải thiện liên tục dựa trên data

### 🚀 Bước tiếp theo:

1. **Notebook 03**: LangFuse Evaluations & Prompts - Đánh giá chất lượng và quản lý prompts
2. **Advanced Tracing**: Tích hợp với production systems
3. **Custom Evaluations**: Tạo evaluation metrics phù hợp với use case
4. **Monitoring Dashboards**: Setup alerting và monitoring
5. **Team Collaboration**: Chia sẻ insights và debugging với team

### 💡 Best Practices:

- Luôn flush traces khi kết thúc session
- Sử dụng meaningful names cho traces và spans
- Thêm metadata và tags để dễ filter và search
- Tạo scores cho các metrics quan trọng
- Regular review traces để identify improvement opportunities

## 📝 Final Cleanup

In [ ]:
# Final flush để đảm bảo tất cả traces được gửi
langfuse.flush()

print("✅ Notebook completed successfully!")
print("🔍 All traces have been sent to LangFuse")
print("🌐 Check your LangFuse dashboard: http://localhost:3000")
print("📊 Analyze your traces and observations for insights!")